In [1]:
%pwd

'c:\\Text-summarization-End-to-End-project\\notebooks'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Text-summarization-End-to-End-project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
# entity 
                                         # frozen means once the instance of the class is created, you cannot change its attributes.
@dataclass(frozen=True)                  # This can be useful for creating objects that should not be changed after they are created.
class DataValidationConfig:
   root_dir: Path
   STATUS_FILE: str
   ALL_REQUIRED_FILES: list

In [6]:
# create a config manager

from src.TextSummarization.constants import *
from src.TextSummarization.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self)-> DataValidationConfig:
        """
        Returns a DataValidationConfig object.
        """
        config = self.config.data_validation
        create_directories([config.root_dir])

        data_configuration_config =  DataValidationConfig(
            root_dir = config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            ALL_REQUIRED_FILES = config.ALL_REQUIRED_FILES
        )
        return data_configuration_config  

In [14]:
import os
from src.TextSummarization.logging.Logging import logger

In [15]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = None

            all_files = os.listdir(os.path.join("artifacts","data_ingestion","samsum_dataset"))

            for file in all_files:

                if file not in self.config.ALL_REQUIRED_FILES:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation_status : {str(validation_status)}")

                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"validation_status : {str(validation_status)}")

            return validation_status

        except Exception as e:
            logger.error(e)
            raise e

In [ ]:
try:
    config = ConfigurationManager()

    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_files_exist()


except Exception as e:
    print(e)